<a href="https://colab.research.google.com/github/gabrielvinicios02/ExperimentsMultivariate/blob/f-TEP/ClassificadorXGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Séries Temporais


<center>
<table><tr><td> <a href="http://www.minds.eng.ufmg.br/"><img src="https://github.com/petroniocandido/pyFTS/raw/master/img/minds_logo_medium.jpeg" alt="MINDS - Machine Intelligence and Data Science Lab" width="100"/></a></td> 
  <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
<td><a href="https://pyfts.github.io/pyFTS/"><img src="https://github.com/petroniocandido/pyFTS/raw/master/img/logo_medium.png" alt="pyFTS - Fuzzy Time Series for Python" width="100"/></a></td>
</tr>
</table>
</center>

In [33]:
!pip3 install -U git+https://github.com/PYFTS/pyFTS
#!pip3 install -U pyFTS
!pip3 install SimpSOM
!pip install dispy
#!pip install matplotlib==3.1.3
!pip install matplotlib==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PYFTS/pyFTS to /tmp/pip-req-build-xziv9ojk
  Running command git clone -q https://github.com/PYFTS/pyFTS /tmp/pip-req-build-xziv9ojk
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.1-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
simpsom 2.0.1 requires matplotlib>=3.3.3, but you have matplotlib 3.1.1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


Dados TEP

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#importing all the necessary libraries 
!pip install pyreadr
import pandas as pd
import seaborn as sns
import numpy as np
import pyreadr as py # library to read .Rdata files in python
import sqlite3
import os
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix,classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer,one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, BatchNormalization,concatenate,Flatten,Embedding,Dense,Dropout,MaxPooling2D,Reshape,BatchNormalization
from keras.models import Sequential
from keras import Model,Input
from keras.layers.convolutional import Conv2D,Conv1D
import keras.backend as k
from sklearn.metrics import roc_auc_score,f1_score
import tensorflow as tf
from keras.callbacks import Callback, EarlyStopping
from time import time
from scipy.sparse import hstack
from tensorflow.python.keras.callbacks import TensorBoard
import pickle
import datetime
from tensorflow.keras.utils import to_categorical
import missingno as msno
import warnings
warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
#reading train data in .R format
a1 = py.read_r("../content/drive/MyDrive/DadosPWFTS/TEP_FaultFree_Training.RData")
a2 = py.read_r("../content/drive/MyDrive/DadosPWFTS/TEP_Faulty_Training.RData")

#reading test data in .R format

#a3 = py.read_r("../content/drive/MyDrive/DadosPWFTS/TEP_FaultFree_Testing.RData")
#a4 = py.read_r("../content/drive/MyDrive/DadosPWFTS/TEP_Faulty_Testing (1).RData")

print("Objects that are present in a1 :",a1.keys())
print("Objects that are present in a2 :",a2.keys())
#print("Objects that are present in a3 :",a3.keys())
#print("Objects that are present in a4 :",a4.keys())

Objects that are present in a1 : odict_keys(['fault_free_training'])
Objects that are present in a2 : odict_keys(['faulty_training'])


In [37]:
#REading the .Rdata files in pandas dataframe and saving it in .csv file
#reading train data
b1 = a1['fault_free_training']
b2 = a2['faulty_training']
#reading test data
#b3 = a3['fault_free_testing']
#b4 = a4['faulty_testing']

# concatinating the train and the test dataset
frames_tr = [b1]
train_ts = pd.concat(frames_tr)
frames_ts = [b2]
test = pd.concat(frames_ts)
#all_data_TEP = [train_ts,test]
#all_TEP = pd.concat(all_data_TEP)

In [38]:
print(test)

         faultNumber  simulationRun  sample  xmeas_1  xmeas_2  xmeas_3  \
0                  1            1.0       1  0.25038   3674.0   4529.0   
1                  1            1.0       2  0.25109   3659.4   4556.6   
2                  1            1.0       3  0.25038   3660.3   4477.8   
3                  1            1.0       4  0.24977   3661.3   4512.1   
4                  1            1.0       5  0.29405   3679.0   4497.0   
...              ...            ...     ...      ...      ...      ...   
4999995           20          500.0     496  0.23419   3655.3   4461.7   
4999996           20          500.0     497  0.26704   3647.4   4540.2   
4999997           20          500.0     498  0.26543   3630.3   4571.6   
4999998           20          500.0     499  0.27671   3655.7   4498.9   
4999999           20          500.0     500  0.27421   3640.4   4474.4   

         xmeas_4  xmeas_5  xmeas_6  xmeas_7  ...   xmv_2   xmv_3   xmv_4  \
0         9.2320   26.889   42.402 

In [39]:
print(test)

         faultNumber  simulationRun  sample  xmeas_1  xmeas_2  xmeas_3  \
0                  1            1.0       1  0.25038   3674.0   4529.0   
1                  1            1.0       2  0.25109   3659.4   4556.6   
2                  1            1.0       3  0.25038   3660.3   4477.8   
3                  1            1.0       4  0.24977   3661.3   4512.1   
4                  1            1.0       5  0.29405   3679.0   4497.0   
...              ...            ...     ...      ...      ...      ...   
4999995           20          500.0     496  0.23419   3655.3   4461.7   
4999996           20          500.0     497  0.26704   3647.4   4540.2   
4999997           20          500.0     498  0.26543   3630.3   4571.6   
4999998           20          500.0     499  0.27671   3655.7   4498.9   
4999999           20          500.0     500  0.27421   3640.4   4474.4   

         xmeas_4  xmeas_5  xmeas_6  xmeas_7  ...   xmv_2   xmv_3   xmv_4  \
0         9.2320   26.889   42.402 

In [40]:
#for i in range(14600000):
  #(test[i]) 

In [41]:
print('Shape of the Train dataset:', train_ts.shape)
print("Shape of the Test dataset:", test.shape)

print("Distrubution of faults in train dataset:")
print(train_ts['faultNumber'].value_counts())

print("Distrubution of faults in test dataset:")
print(test['faultNumber'].value_counts())

Shape of the Train dataset: (250000, 55)
Shape of the Test dataset: (5000000, 55)
Distrubution of faults in train dataset:
0.0    250000
Name: faultNumber, dtype: int64
Distrubution of faults in test dataset:
1     250000
2     250000
19    250000
18    250000
17    250000
16    250000
15    250000
14    250000
13    250000
12    250000
11    250000
10    250000
9     250000
8     250000
7     250000
6     250000
5     250000
4     250000
3     250000
20    250000
Name: faultNumber, dtype: int64


In [42]:
train_ts_2 = train_ts[0:50000]
test_2 = test[0:50000]

In [43]:
print(train_ts_2)

       faultNumber  simulationRun  sample  xmeas_1  xmeas_2  xmeas_3  xmeas_4  \
0              0.0            1.0       1  0.25038   3674.0   4529.0   9.2320   
1              0.0            1.0       2  0.25109   3659.4   4556.6   9.4264   
2              0.0            1.0       3  0.25038   3660.3   4477.8   9.4426   
3              0.0            1.0       4  0.24977   3661.3   4512.1   9.4776   
4              0.0            1.0       5  0.29405   3679.0   4497.0   9.3381   
...            ...            ...     ...      ...      ...      ...      ...   
49995          0.0          100.0     496  0.22895   3687.0   4589.8   9.3720   
49996          0.0          100.0     497  0.23682   3609.3   4423.7   9.4411   
49997          0.0          100.0     498  0.23891   3557.8   4481.0   9.4321   
49998          0.0          100.0     499  0.24640   3647.8   4452.3   9.4553   
49999          0.0          100.0     500  0.24346   3672.7   4471.3   9.4429   

       xmeas_5  xmeas_6  xm

In [44]:
print(test)# com falhas

         faultNumber  simulationRun  sample  xmeas_1  xmeas_2  xmeas_3  \
0                  1            1.0       1  0.25038   3674.0   4529.0   
1                  1            1.0       2  0.25109   3659.4   4556.6   
2                  1            1.0       3  0.25038   3660.3   4477.8   
3                  1            1.0       4  0.24977   3661.3   4512.1   
4                  1            1.0       5  0.29405   3679.0   4497.0   
...              ...            ...     ...      ...      ...      ...   
4999995           20          500.0     496  0.23419   3655.3   4461.7   
4999996           20          500.0     497  0.26704   3647.4   4540.2   
4999997           20          500.0     498  0.26543   3630.3   4571.6   
4999998           20          500.0     499  0.27671   3655.7   4498.9   
4999999           20          500.0     500  0.27421   3640.4   4474.4   

         xmeas_4  xmeas_5  xmeas_6  xmeas_7  ...   xmv_2   xmv_3   xmv_4  \
0         9.2320   26.889   42.402 

In [45]:
print(train_ts)# sem falhas

        faultNumber  simulationRun  sample  xmeas_1  xmeas_2  xmeas_3  \
0               0.0            1.0       1  0.25038   3674.0   4529.0   
1               0.0            1.0       2  0.25109   3659.4   4556.6   
2               0.0            1.0       3  0.25038   3660.3   4477.8   
3               0.0            1.0       4  0.24977   3661.3   4512.1   
4               0.0            1.0       5  0.29405   3679.0   4497.0   
...             ...            ...     ...      ...      ...      ...   
249995          0.0          500.0     496  0.29325   3640.1   4473.0   
249996          0.0          500.0     497  0.29134   3625.7   4506.2   
249997          0.0          500.0     498  0.29438   3600.2   4478.3   
249998          0.0          500.0     499  0.25269   3683.5   4486.4   
249999          0.0          500.0     500  0.25214   3648.2   4467.8   

        xmeas_4  xmeas_5  xmeas_6  xmeas_7  ...   xmv_2   xmv_3   xmv_4  \
0        9.2320   26.889   42.402   2704.3  ... 

In [46]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from pyFTS.models import hofts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

In [47]:
from pyFTS.data import TAIEX, Malaysia, sunspots, artificial
from statsmodels.tsa.stattools import acf, pacf

datasets = {}

dff = pd.read_excel('../content/drive/MyDrive/DadosPWFTS/DadosFalha.xlsx')
dfn = pd.read_excel('../content/drive/MyDrive/DadosPWFTS/DadosNormais.xlsx')

dados_treino_Falha = dff[0:2300] 
dados_teste_Falha = dff[1800:2300]

dados_treino_Normal = dfn[0:1800] 
dados_teste_Normal = dfn[1500:1800]

In [48]:
print(dados_treino_Falha)

                    Data  LIT_2012KS_0251_FilteredSignal  \
0    2021-12-05 08:50:13                       35.298019   
1    2021-12-05 08:51:25                       69.698112   
2    2021-12-05 08:52:37                       86.492775   
3    2021-12-05 08:53:49                       87.605766   
4    2021-12-05 08:55:01                       87.592270   
...                  ...                             ...   
2295 2021-12-09 22:55:01                       39.161282   
2296 2021-12-09 22:56:13                       34.402630   
2297 2021-12-09 22:57:25                       29.267023   
2298 2021-12-09 22:58:37                       24.829168   
2299 2021-12-09 22:59:49                       18.879450   

      _AL_2012KS_06M1_SpeedActual  WIT_1085KS_2300_BeltLoad  \
0                        0.000000                604.000305   
1                        0.000000                594.372681   
2                        0.000000                548.985413   
3                        0.

In [49]:
print(dados_treino_Normal)

                    Data  LIT_2012KS_0251_FilteredSignal  \
0    2021-12-05 07:20:13                        2.832069   
1    2021-12-05 07:21:25                        4.418116   
2    2021-12-05 07:22:37                       22.953905   
3    2021-12-05 07:23:49                       33.815975   
4    2021-12-05 07:25:01                       33.374435   
...                  ...                             ...   
1795 2021-12-07 02:38:13                       11.774282   
1796 2021-12-07 02:39:25                        4.781181   
1797 2021-12-07 02:40:37                        3.563411   
1798 2021-12-07 02:41:49                        3.671766   
1799 2021-12-07 02:43:01                        3.691521   

      _AL_2012KS_06M1_SpeedActual  WIT_1085KS_2300_BeltLoad  \
0                        0.000000                295.496826   
1                        0.000000                321.693665   
2                       14.761859                337.279755   
3                       71.

In [50]:
print(dados_teste_Falha)

                    Data  LIT_2012KS_0251_FilteredSignal  \
1800 2021-12-08 22:32:13                       68.584373   
1801 2021-12-08 22:33:25                       79.267204   
1802 2021-12-08 22:34:37                       74.231430   
1803 2021-12-08 22:35:49                       67.143143   
1804 2021-12-08 22:37:01                       56.879524   
...                  ...                             ...   
2295 2021-12-09 22:55:01                       39.161282   
2296 2021-12-09 22:56:13                       34.402630   
2297 2021-12-09 22:57:25                       29.267023   
2298 2021-12-09 22:58:37                       24.829168   
2299 2021-12-09 22:59:49                       18.879450   

      _AL_2012KS_06M1_SpeedActual  WIT_1085KS_2300_BeltLoad  \
1800                    36.922737                551.704773   
1801                    51.525627                551.704529   
1802                    59.430199                551.716675   
1803                    59.

In [51]:
print(dados_teste_Normal)

                    Data  LIT_2012KS_0251_FilteredSignal  \
1500 2021-12-08 09:28:37                       43.040955   
1501 2021-12-08 09:29:49                       37.851479   
1502 2021-12-08 09:31:01                       32.487022   
1503 2021-12-08 09:32:13                       27.220686   
1504 2021-12-08 09:33:25                       22.106337   
...                  ...                             ...   
1795 2021-12-07 02:38:13                       11.774282   
1796 2021-12-07 02:39:25                        4.781181   
1797 2021-12-07 02:40:37                        3.563411   
1798 2021-12-07 02:41:49                        3.671766   
1799 2021-12-07 02:43:01                        3.691521   

      _AL_2012KS_06M1_SpeedActual  WIT_1085KS_2300_BeltLoad  \
1500                    59.528919                297.226349   
1501                    59.413948                246.993668   
1502                    59.192154                220.550995   
1503                    59.

## Analisando as Séries Temporais

In [52]:
#Dados sem Falhas

from pyFTS.data import TAIEX, Malaysia, sunspots, artificial
from statsmodels.tsa.stattools import acf, pacf

datasets1 = {}

#datasets1['LIT'] = dados_treino_Falha.get('LIT_2012KS_0251_FilteredSignal')
datasets1['LIT'] = train_ts_2.get('xmeas_6')
#datasets1['SpeedActual'] = dados_treino_Falha.get('_AL_2012KS_06M1_SpeedActual')
datasets1['SpeedActual'] = train_ts_2.get('xmeas_7')
#datasets1['BeltLoad'] = dados_treino_Falha.get('WIT_1085KS_2300_BeltLoad')
datasets1['BeltLoad'] = train_ts_2.get('xmeas_8')
#datasets1['Rate'] = dados_treino_Falha.get('WIT_1085KS_2300_Rate')
datasets1['Rate'] = train_ts_2.get('xmeas_9')

In [53]:
##Dados sem Falhas

#from pyFTS.data import TAIEX, Malaysia, sunspots, artificial
#from statsmodels.tsa.stattools import acf, pacf
#
#datasets1 = {}
#
##datasets1['LIT'] = dados_treino_Falha.get('LIT_2012KS_0251_FilteredSignal')
#datasets1['LIT'] = train_ts.get('xmeas_6')
##datasets1['SpeedActual'] = dados_treino_Falha.get('_AL_2012KS_06M1_SpeedActual')
#datasets1['SpeedActual'] = train_ts.get('xmeas_7')
##datasets1['BeltLoad'] = dados_treino_Falha.get('WIT_1085KS_2300_BeltLoad')
#datasets1['BeltLoad'] = train_ts.get('xmeas_8')
##datasets1['Rate'] = dados_treino_Falha.get('WIT_1085KS_2300_Rate')
#datasets1['Rate'] = train_ts.get('xmeas_9')

#fig1, ax = plt.subplots(nrows=4, ncols=4, figsize=[20,10])

#for ct, (key, data) in enumerate(datasets1.items()): 
#  print(ct)
#  print(data)
#  _lags = np.arange(0,101)
#  _acf = acf(data,nlags=100)
#  _pacf = pacf(data,nlags=100)
#  
#  ax[0][ct].set_title("{}".format(key))
#  ax[0][ct].plot(data)
#  
#  ax[1][ct].set_title("{} 100 intances".format(key))
#  ax[1][ct].plot(data[:100])
#  
#  ax[2][ct].set_title("ACF {} - 100 lags".format(key))
#  ax[2][ct].plot(_lags, _acf, "o")
#  ax[2][ct].vlines(_lags,[0],_acf)
#  ax[2][ct].axhline(y=0, color='black')
#  
#  ax[3][ct].set_title("PACF {} - 100 lags".format(key))
#  ax[3][ct].plot(_lags, _pacf, "o")
#  ax[3][ct].vlines(_lags,[0],_pacf)
#  ax[3][ct].axhline(y=0, color='black')
#
#plt.tight_layout()

In [54]:
#Dados normais - com falhas
from pyFTS.data import TAIEX, Malaysia, sunspots, artificial
from statsmodels.tsa.stattools import acf, pacf

datasets2 = {}

#datasets2['LIT'] = dados_treino_Normal.get('LIT_2012KS_0251_FilteredSignal')
datasets2['LIT'] = test_2.get('xmeas_6')
#datasets2['SpeedActual'] = dados_treino_Normal.get('_AL_2012KS_06M1_SpeedActual')
datasets2['SpeedActual'] = test_2.get('xmeas_7')
#datasets2['BeltLoad'] = dados_treino_Normal.get('WIT_1085KS_2300_BeltLoad')
datasets2['BeltLoad'] = test_2.get('xmeas_8')
#datasets2['Rate'] = dados_treino_Normal.get('WIT_1085KS_2300_Rate')
datasets2['Rate'] = test_2.get('xmeas_9')

In [55]:
##Dados normais - sem falhas
#from pyFTS.data import TAIEX, Malaysia, sunspots, artificial
#from statsmodels.tsa.stattools import acf, pacf

#datasets2 = {}

##datasets2['LIT'] = dados_treino_Normal.get('LIT_2012KS_0251_FilteredSignal')
#datasets2['LIT'] = test.get('xmeas_6')
##datasets2['SpeedActual'] = dados_treino_Normal.get('_AL_2012KS_06M1_SpeedActual')
##datasets2['SpeedActual'] = test.get('xmeas_7')
##datasets2['BeltLoad'] = dados_treino_Normal.get('WIT_1085KS_2300_BeltLoad')
#datasets2['BeltLoad'] = test.get('xmeas_8')
##datasets2['Rate'] = dados_treino_Normal.get('WIT_1085KS_2300_Rate')
#datasets2['Rate'] = test.get('xmeas_9')


#fig2, ax = plt.subplots(nrows=4, ncols=4, figsize=[20,10])

#for ct, (key, data) in enumerate(datasets2.items()):
#  print(ct)
#  print(data)
#  _lags = np.arange(0,101)
#  _acf = acf(data,nlags=100)
#  _pacf = pacf(data,nlags=100)
#  
#  ax[0][ct].set_title("{}".format(key))
#  ax[0][ct].plot(data)
#  
#  ax[1][ct].set_title("{} 100 intances".format(key))
#  ax[1][ct].plot(data[:100])
#  
#  ax[2][ct].set_title("ACF {} - 100 lags".format(key))
#  ax[2][ct].plot(_lags, _acf, "o")
#  ax[2][ct].vlines(_lags,[0],_acf)
#  ax[2][ct].axhline(y=0, color='black')
#  
#  ax[3][ct].set_title("PACF {} - 100 lags".format(key))
#  ax[3][ct].plot(_lags, _pacf, "o")
#  ax[3][ct].vlines(_lags,[0],_pacf)
#  ax[3][ct].axhline(y=0, color='black')

#plt.tight_layout()

In [56]:
##Dados normais - sem falhas
#from pyFTS.data import TAIEX, Malaysia, sunspots, artificial
#from statsmodels.tsa.stattools import acf, pacf

#datasets3 = {}

#datasets3['LIT'] = dados_teste_Normal.get('LIT_2012KS_0251_FilteredSignal')
#datasets3['SpeedActual'] = dados_teste_Normal.get('_AL_2012KS_06M1_SpeedActual')
#datasets3['BeltLoad'] = dados_teste_Normal.get('WIT_1085KS_2300_BeltLoad')
#datasets3['Rate'] = dados_teste_Normal.get('WIT_1085KS_2300_Rate')


#fig3, ax = plt.subplots(nrows=4, ncols=4, figsize=[20,10])

#for ct, (key, data) in enumerate(datasets3.items()): 
#  _lags = np.arange(0,101)
#  _acf = acf(data,nlags=100)
#  _pacf = pacf(data,nlags=100)
  
#  ax[0][ct].set_title("{}".format(key))
#  ax[0][ct].plot(data)
  
#  ax[1][ct].set_title("{} 100 intances".format(key))
#  ax[1][ct].plot(data[:100])
  
#  ax[2][ct].set_title("ACF {} - 100 lags".format(key))
#  ax[2][ct].plot(_lags, _acf, "o")
#  ax[2][ct].vlines(_lags,[0],_acf)
#  ax[2][ct].axhline(y=0, color='black')
  
#  ax[3][ct].set_title("PACF {} - 100 lags".format(key))
#  ax[3][ct].plot(_lags, _pacf, "o")
#  ax[3][ct].vlines(_lags,[0],_pacf)
#  ax[3][ct].axhline(y=0, color='black')

#plt.tight_layout()

In [57]:
#verifica diferenca do grid
#from pyFTS.models import hofts
#from pyFTS.partitioners import Grid, FCM, CMeans, Entropy
#from pyFTS.benchmarks import Measures

#metodos = [Grid.GridPartitioner, Entropy.EntropyPartitioner, CMeans.CMeansPartitioner ]

#k = 30

#rows = []

#y1=np.asarray(datasets1['SpeedActual'])

#fig, ax = plt.subplots(nrows=1, ncols=1,figsize=[15,5])

#ax.plot(y1, label='Original',color='black')

#for contador, metodo in enumerate(metodos):
#  part = metodo(data=y1, npart=k)
#  model = hofts.HighOrderFTS(order=2, partitioner=part)
#  model.fit(y1)
#  forecasts = model.predict(y1)
#  for o in range(model.order):
#    forecasts.insert(0,None)
    
#  ax.plot(forecasts[:-1], label=part.name)
  
#  rmse, mape, u = Measures.get_point_statistics(y1, model)
  
#  rows.append([part.name, rmse, mape, u])
  

#handles, labels = ax.get_legend_handles_labels()
#lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))

#pd.DataFrame(rows, columns=['Partitions','RMSE','MAPE','U'])

In [58]:
#Dados com Falhas
from numpy.core.multiarray import concatenate
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures
from pyFTS.common import Util

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y1 = datasets2['LIT']
tam_grupo = 10000
i=0
VT1=[]

for i in range(0,y1.size,tam_grupo):
  grpan = y1[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model1 = metodo(partitioner=part, progress=False, type='distribution')
    if model1.is_high_order:
      model1 = metodo(partitioner=part, order=1)
    model1.fit(np.asarray(y1))  
  #print(model1)
  _, _, lhs1, rhs1 = pwfts.highorder_fuzzy_markov_chain(model1)
  lhs1t = np.transpose(lhs1)
  V1 = np.c_[rhs1,lhs1t]
  if i == 0 :
    VT1 = V1
  else:
    VT1 = np.c_[VT1,V1]

  print(i)

VT1 = np.transpose(VT1)
print(VT1)

0


ValueError: ignored

In [62]:
#Dados com Falhas
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y2 = datasets2['SpeedActual']
tam_grupo = 10000
i=0
VT2=[]
V2=[]

for i in range(0,y2.size,tam_grupo):
  grpan = y2[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model2 = metodo(partitioner=part, progress=False, type='distribution')
    if model2.is_high_order:
      model2 = metodo(partitioner=part, order=1)
    model2.fit(np.asarray(y2))  
  #print(model1)
  _, _, lhs2, rhs2 = pwfts.highorder_fuzzy_markov_chain(model2)
  lhs2t = np.transpose(lhs2)
  V2 = np.c_[rhs2,lhs2t]
  if i == 0 :
    VT2 = V2
  else:
    VT2 = np.c_[VT2,V2]

  print(i)

VT2 = np.transpose(VT2)
print(VT2)



0
10000
20000
30000
40000
[[1.20846414e-01 9.93240589e-04 7.51423052e-07]
 [8.78590975e-01 6.95819467e-01 5.85569288e-01]
 [5.62610965e-04 3.03187293e-01 4.14429961e-01]
 [7.44123121e-04 6.58392725e-01 3.40863152e-01]
 [1.25366364e-01 1.22112890e-03 9.02247021e-07]
 [8.74111892e-01 7.08361683e-01 5.92523453e-01]
 [5.21743563e-04 2.90417188e-01 4.07475645e-01]
 [9.36423022e-04 6.70470249e-01 3.28593328e-01]
 [1.06003084e-01 3.64645929e-04 9.24837063e-07]
 [8.93572462e-01 6.42224884e-01 5.52002078e-01]
 [4.24454207e-04 3.57410470e-01 4.47996997e-01]
 [2.47933432e-04 6.06858168e-01 3.92893898e-01]
 [1.35177239e-01 1.71380770e-03 1.24305019e-06]
 [8.64358257e-01 7.31008585e-01 6.03910631e-01]
 [4.64504435e-04 2.67277607e-01 3.96088126e-01]
 [1.37230316e-03 6.92270312e-01 3.06357385e-01]
 [1.45447122e-01 2.36926069e-03 1.40890018e-06]
 [8.53942378e-01 7.54376642e-01 6.13759096e-01]
 [6.10499627e-04 2.43254097e-01 3.86239495e-01]
 [1.98214353e-03 7.14757721e-01 2.83260136e-01]]


In [64]:
#Dados com Falhas
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y3 = datasets2['BeltLoad']
tam_grupo = 10000
i=0
VT3=[]
V3=[]

for i in range(0,y3.size,tam_grupo):
  grpan = y3[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model3 = metodo(partitioner=part, progress=False, type='distribution')
    if model3.is_high_order:
      model3 = metodo(partitioner=part, order=1)
    model3.fit(np.asarray(y3))  
  #print(model1)
  _, _, lhs3, rhs3 = pwfts.highorder_fuzzy_markov_chain(model3)
  lhs3t = np.transpose(lhs3)
  V3 = np.c_[rhs3,lhs3t]
  if i == 0 :
    VT3 = V3
  else:
    VT3 = np.c_[VT3,V3]

  print(i)

VT3 = np.transpose(VT3)
print(VT3)

0
10000
20000
30000
40000
[[2.05747777e-01 2.09494646e-03 7.18228247e-06]
 [7.92314635e-01 6.81490029e-01 6.45947535e-01]
 [1.93758807e-03 3.16415024e-01 3.54045283e-01]
 [1.77023827e-03 6.70021130e-01 3.28208632e-01]
 [2.16402890e-01 8.70712442e-04 3.11315528e-06]
 [7.80365273e-01 5.46951269e-01 5.08634271e-01]
 [3.23183734e-03 4.52178018e-01 4.91362616e-01]
 [5.89753476e-04 5.29066595e-01 4.70343652e-01]
 [1.83155991e-01 4.23132585e-04 1.08850170e-06]
 [8.14133143e-01 4.57765810e-01 4.21140780e-01]
 [2.71086565e-03 5.41811057e-01 5.78858132e-01]
 [2.27245922e-04 4.37243935e-01 5.62528819e-01]
 [2.33473456e-01 1.13058836e-03 4.47986931e-06]
 [7.63641192e-01 5.44187543e-01 5.02050667e-01]
 [2.88535218e-03 4.54681869e-01 4.97944853e-01]
 [7.76162022e-04 5.24347705e-01 4.74876133e-01]
 [2.35520517e-01 2.72564456e-03 1.05310630e-05]
 [7.62254190e-01 6.66924837e-01 6.25502547e-01]
 [2.22529322e-03 3.30349519e-01 3.74486922e-01]
 [2.33244878e-03 6.52865061e-01 3.44802490e-01]]


In [65]:
#Dados com Falhas
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y4 = datasets2['Rate']
tam_grupo = 10000
i=0
VT4=[]
V4=[]

for i in range(0,y4.size,tam_grupo):
  grpan = y4[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model4 = metodo(partitioner=part, progress=False, type='distribution')
    if model4.is_high_order:
      model4 = metodo(partitioner=part, order=1)
    model4.fit(np.asarray(y4))  
  #print(model1)
  _, _, lhs4, rhs4 = pwfts.highorder_fuzzy_markov_chain(model4)
  lhs4t = np.transpose(lhs4)
  V4 = np.c_[rhs4,lhs4t]
  if i == 0 :
    VT4 = V4
  else:
    VT4 = np.c_[VT4,V4]

  print(i)

VT4 = np.transpose(VT4)
print(VT4)

0
10000
20000
30000
40000
[[0.         0.        ]
 [0.49251272 0.49242321]
 [0.50748728 0.50757679]
 [0.50753141 0.49246859]
 [0.         0.        ]
 [0.49715539 0.49706581]
 [0.50284461 0.50293419]
 [0.50288919 0.49711081]
 [0.         0.        ]
 [0.48880984 0.48872011]
 [0.51119016 0.51127989]
 [0.51123406 0.48876594]
 [0.         0.        ]
 [0.48893779 0.48884821]
 [0.51106221 0.51115179]
 [0.51110605 0.48889395]
 [0.         0.        ]
 [0.49223005 0.49214089]
 [0.50776995 0.50785911]
 [0.50781388 0.49218612]]


In [66]:
#Dados sem Falhas
from numpy.core.multiarray import concatenate
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures
from pyFTS.common import Util

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y1 = datasets1['LIT']
tam_grupo = 10000
i=0
VU1=[]

for i in range(0,y1.size,tam_grupo):
  grpan = y1[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model1 = metodo(partitioner=part, progress=False, type='distribution')
    if model1.is_high_order:
      model1 = metodo(partitioner=part, order=1)
    model1.fit(np.asarray(y1))  
  #print(model1)
  _, _, lhs1, rhs1 = pwfts.highorder_fuzzy_markov_chain(model1)
  lhs1t = np.transpose(lhs1)
  V1 = np.c_[rhs1,lhs1t]
  if i == 0 :
    VU1 = V1
  else:
    VU1 = np.c_[VU1,V1]

  print(i)

VU1 = np.transpose(VU1)
print(VU1)

0
10000
20000
30000
40000
[[0.         0.        ]
 [0.54483982 0.54435304]
 [0.45516018 0.45564696]
 [0.54461753 0.45538247]
 [0.         0.        ]
 [0.51461544 0.51411891]
 [0.48538456 0.48588109]
 [0.51437369 0.48562631]
 [0.         0.        ]
 [0.52163822 0.52114069]
 [0.47836178 0.47885931]
 [0.52139947 0.47860053]
 [0.         0.        ]
 [0.52732017 0.526819  ]
 [0.47267983 0.473181  ]
 [0.52708253 0.47291747]
 [0.         0.        ]
 [0.51825494 0.51776769]
 [0.48174506 0.48223231]
 [0.51801948 0.48198052]]


In [67]:
#Dados sem Falhas
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y2 = datasets1['SpeedActual']
tam_grupo = 10000
i=0
VU2=[]
V2=[]

for i in range(0,y2.size,tam_grupo):
  grpan = y2[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model2 = metodo(partitioner=part, progress=False, type='distribution')
    if model2.is_high_order:
      model2 = metodo(partitioner=part, order=1)
    model2.fit(np.asarray(y2))  
  #print(model1)
  _, _, lhs2, rhs2 = pwfts.highorder_fuzzy_markov_chain(model2)
  lhs2t = np.transpose(lhs2)
  V2 = np.c_[rhs2,lhs2t]
  if i == 0 :
    VU2 = V2
  else:
    VU2 = np.c_[VU2,V2]

  print(i)

VU2 = np.transpose(VU2)
print(VU2)

0
10000
20000
30000
40000
[[0.         0.        ]
 [0.51626768 0.51037267]
 [0.48373232 0.48962733]
 [0.51340048 0.48659952]
 [0.         0.        ]
 [0.5393191  0.53356977]
 [0.4606809  0.46643023]
 [0.53665647 0.46334353]
 [0.         0.        ]
 [0.51311841 0.50718898]
 [0.48688159 0.49281102]
 [0.51021558 0.48978442]
 [0.         0.        ]
 [0.52058116 0.5146666 ]
 [0.47941884 0.4853334 ]
 [0.51773005 0.48226995]
 [0.         0.        ]
 [0.53432887 0.52857981]
 [0.46567113 0.47142019]
 [0.53163752 0.46836248]]


In [68]:
#Dados sem Falhas
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y3 = datasets1['BeltLoad']
tam_grupo = 10000
i=0
VU3=[]
V3=[]

for i in range(0,y3.size,tam_grupo):
  grpan = y3[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model3 = metodo(partitioner=part, progress=False, type='distribution')
    if model3.is_high_order:
      model3 = metodo(partitioner=part, order=1)
    model3.fit(np.asarray(y3))  
  #print(model1)
  _, _, lhs3, rhs3 = pwfts.highorder_fuzzy_markov_chain(model3)
  lhs3t = np.transpose(lhs3)
  V3 = np.c_[rhs3,lhs3t]
  if i == 0 :
    VU3 = V3
  else:
    VU3 = np.c_[VU3,V3]

  print(i)

VU3 = np.transpose(VU3)
print(VU3)

0
10000
20000
30000
40000
[[0.         0.        ]
 [0.57113009 0.56666728]
 [0.42886991 0.43333272]
 [0.56920918 0.43079082]
 [0.         0.        ]
 [0.50629674 0.50151426]
 [0.49370326 0.49848574]
 [0.50392598 0.49607402]
 [0.         0.        ]
 [0.51346217 0.50868322]
 [0.48653783 0.49131678]
 [0.51112757 0.48887243]
 [0.         0.        ]
 [0.53342793 0.52879188]
 [0.46657207 0.47120812]
 [0.53125648 0.46874352]
 [0.         0.        ]
 [0.54087504 0.53618285]
 [0.45912496 0.46381715]
 [0.53871227 0.46128773]]


In [69]:
#Dados sem Falhas
from pyFTS.models import pwfts
#from pyFTS.models import chen, cheng, hofts, pwfts
from pyFTS.partitioners import Grid
from pyFTS.benchmarks import Measures

#metodos = [chen.ConventionalFTS, cheng.TrendWeightedFTS, hofts.HighOrderFTS, 
#          hofts.WeightedHighOrderFTS, pwfts.ProbabilisticWeightedFTS]
metodos = [pwfts.ProbabilisticWeightedFTS]

k = 3
y4 = datasets1['Rate']
tam_grupo = 10000
i=0
VU4=[]
V4=[]

for i in range(0,y4.size,tam_grupo):
  grpan = y4[i:i+tam_grupo]
  part = Grid.GridPartitioner(data=np.asarray(grpan), npart=k)
  for contador, metodo in enumerate(metodos):
    model4 = metodo(partitioner=part, progress=False, type='distribution')
    if model4.is_high_order:
      model4 = metodo(partitioner=part, order=1)
    model4.fit(np.asarray(y4))  
  #print(model1)
  _, _, lhs4, rhs4 = pwfts.highorder_fuzzy_markov_chain(model4)
  lhs4t = np.transpose(lhs4)
  V4 = np.c_[rhs4,lhs4t]
  if i == 0 :
    VU4 = V4
  else:
    VU4 = np.c_[VU4,V4]

  print(i)

VU4 = np.transpose(VU4)
print(VU4)

0
10000
20000
30000
40000
[[0.         0.        ]
 [0.5003089  0.50030534]
 [0.4996911  0.49969466]
 [0.49969293 0.50030707]
 [0.         0.        ]
 [0.50154696 0.50154341]
 [0.49845304 0.49845659]
 [0.49845487 0.50154513]
 [0.         0.        ]
 [0.5008664  0.50086285]
 [0.4991336  0.49913715]
 [0.49913543 0.50086457]
 [0.         0.        ]
 [0.5008664  0.50086285]
 [0.4991336  0.49913715]
 [0.49913543 0.50086457]
 [0.         0.        ]
 [0.4997518  0.49974825]
 [0.5002482  0.50025175]
 [0.50025002 0.49974998]]


In [ ]:
ytst1 = np.ones(360)
ytst2 = np.zeros(253)
ytst = np.concatenate((ytst1,ytst2))

In [ ]:
#Unir todas as matrizes geradas
BaseT1 = np.transpose(np.concatenate([np.transpose(VT1),np.transpose(VT2),np.transpose(VT3),np.transpose(VT4)]))
BaseT2 = np.transpose(np.concatenate([np.transpose(VU1),np.transpose(VU2),np.transpose(VU3),np.transpose(VU4)]))
BaseT3 = np.c_[np.transpose(BaseT1), np.transpose(BaseT2)]
BaseT4 = np.c_[np.transpose(BaseT3),ytst]

In [ ]:
pwfts.visualize_distributions(model1)

In [ ]:
pwfts.visualize_distributions(model2)

In [ ]:
pwfts.visualize_distributions(model3)

In [ ]:
pwfts.visualize_distributions(model4)

In [ ]:
#dataset = pd.read_excel('../content/sample_data/Base.xlsx')
#x = dataset.iloc[:,:-1].values
#y = dataset.iloc[:,-1].values
x = BaseT4[:,:-1]
y = BaseT4[:,-1]

In [ ]:
import seaborn as sns
#dataset2 = pd.read_excel('../content/sample_data/Base.xlsx')
corr_df = dff.corr(method='pearson')

plt.figure(figsize=(8, 6))
sns.heatmap(corr_df, annot=True)
plt.show()

In [ ]:
import seaborn as sns
#dataset2 = pd.read_excel('../content/sample_data/Base.xlsx')
corr_df = dfn.corr(method='pearson')

plt.figure(figsize=(8, 6))
sns.heatmap(corr_df, annot=True)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size = 0.2, random_state = 10)

In [ ]:
from xgboost import XGBClassifier
Classifier = XGBClassifier()
Classifier.fit(x_train, y_train)

Matriz de confusão

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
ypred = Classifier.predict(x_test)
cm = confusion_matrix(y_test, ypred)
print(cm)

Precisão

In [ ]:
accuracy_score(y_test, ypred)

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = Classifier, X = x_train, y = y_train, cv = 2)

print(accuracies.mean()*100)
print(accuracies.std()*100)